<a href="https://colab.research.google.com/github/JRMJ14/JRMJ14/blob/main/clima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 8.8 MB/s eta 0:00:00


**Rutas de navegacion**

juego donde puedes implementar un entorno que simule diferentes condiciones climáticas donde el agente deba tomar decisiones sobre cómo navegar, por ejemplo, rutas de navegación por aire o mar que eviten condiciones peligrosas.

In [2]:
#importamos las respectivas librerias necesarias para el desarrollo del entorno
import gymnasium as gym
import numpy as np
import random

In [3]:
class NavigationEnv(gym.Env):
    def __init__(self):
        super(NavigationEnv, self).__init__()

        # Tamaño del entorno de navegación (10x10)
        self.grid_size = 10

        # Espacio de observación: todas las posiciones posibles (10x10 = 100 estados)
        self.observation_space = gym.spaces.Discrete(self.grid_size ** 2)

        # Espacio de acción: arriba, abajo, izquierda, derecha
        self.action_space = gym.spaces.Discrete(4)

        # Definir punto de inicio y destino
        self.start_position = (0, 0)
        self.goal_position = (9, 9)

        # Zonas de clima adverso (ubicaciones peligrosas)
        self.bad_weather_zones = [(3, 3), (4, 5), (6, 6), (7, 2), (8, 8)]

        # Estado inicial del agente
        self.state = self.start_position

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.state = (random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1))  # Reiniciar el agente al inicio
        #self.state = self.start_position  # Reiniciar el agente al inicio
        return self._get_state_index(self.state), {}

    def step(self, action):
        x, y = self.state

        # Movimiento basado en la acción seleccionada
        if action == 0 and y > 0:          # Arriba
            y -= 1
        elif action == 1 and y < self.grid_size - 1:  # Abajo
            y += 1
        elif action == 2 and x > 0:        # Izquierda
            x -= 1
        elif action == 3 and x < self.grid_size - 1:  # Derecha
            x += 1

        # Actualizar el estado del agente
        self.state = (x, y)

        # Recompensas y finalización
        reward = -1  # Penalización por cada paso
        done = False

        # Si alcanza el destino, gran recompensa
        if self.state == self.goal_position:
            reward = 20
            done = True
        # Penalización adicional por entrar en clima peligroso
        elif self.state in self.bad_weather_zones:
            reward = -10

        return self._get_state_index(self.state), reward, done, False, {}

    def _get_state_index(self, position):
        """Convierte una posición (x, y) a un índice único para la Q-table."""
        return position[0] * self.grid_size + position[1]

In [4]:
# Crear el entorno
env = NavigationEnv()

In [8]:
# Parámetros de aprendizaje Q-Learning
q_table = np.zeros([env.observation_space.n, env.action_space.n])  # Tabla Q inicial
alpha = 0.1       # Tasa de aprendizaje
gamma = 0.9       # Factor de descuento
epsilon = 1.0     # Factor de exploración inicial
epsilon_decay = 0.800  # Decaimiento de epsilon para reducir la exploración
min_epsilon = 0.01

In [9]:
# Entrenamiento del agente
num_episodes = 1000
for episode in range(num_episodes):
    state, _ = env.reset()
    done = False

    while not done:
        # Elegir acción (exploración vs explotación)
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Exploración
        else:
            action = np.argmax(q_table[state])  # Explotación

        # Ejecutar la acción en el entorno
        next_state, reward, done, _, _ = env.step(action)

        # Actualizar la Q-table
        best_next_action = np.argmax(q_table[next_state])
        td_target = reward + gamma * q_table[next_state, best_next_action]
        td_error = td_target - q_table[state, action]
        q_table[state, action] += alpha * td_error

        # Avanzar al siguiente estado
        state = next_state

    # Decaer epsilon después de cada episodio
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

print("Entrenamiento completado.")

Entrenamiento completado.


In [11]:
# Evaluación del agente
test_episodes = 10
for episode in range(test_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = np.argmax(q_table[state])  # Elegir la mejor acción
        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        state = next_state
    print(f"Recompensa total en episodio de prueba {episode + 1}: {total_reward}")

Recompensa total en episodio de prueba 1: 9
Recompensa total en episodio de prueba 2: 14
Recompensa total en episodio de prueba 3: 5
Recompensa total en episodio de prueba 4: 6
Recompensa total en episodio de prueba 5: 18
Recompensa total en episodio de prueba 6: 12
Recompensa total en episodio de prueba 7: 19
Recompensa total en episodio de prueba 8: 16
Recompensa total en episodio de prueba 9: 12
Recompensa total en episodio de prueba 10: 6


In [ ]:
print("Tabla Q después del entrenamiento:")
print(q_table)

Tabla Q después del entrenamiento:
[[-5.81836833e+00 -4.99684549e+00 -6.30964907e+00 -6.30707489e+00]
 [-5.73106759e+00 -4.44093943e+00 -5.23545757e+00 -5.71700325e+00]
 [-5.35528875e+00 -3.82326604e+00 -5.18886588e+00 -4.65676547e+00]
 [-4.92725472e+00 -4.01325454e+00 -4.33895207e+00 -3.13696226e+00]
 [-4.65060356e+00 -4.74019778e+00 -4.76424456e+00 -2.59452468e+00]
 [-4.50186927e+00 -4.41794328e+00 -4.43458822e+00 -4.40599285e+00]
 [-4.12874081e+00 -4.06748588e+00 -4.15102218e+00 -4.05967182e+00]
 [-3.77560280e+00 -3.73580092e+00 -3.77329459e+00 -3.70261341e+00]
 [-3.48505568e+00 -3.47459766e+00 -3.46577926e+00 -3.51692962e+00]
 [-3.31613058e+00 -3.29183187e+00 -3.27395769e+00 -3.32670318e+00]
 [-5.99161165e+00 -5.68505572e+00 -6.13966109e+00 -5.96194583e+00]
 [-5.65853290e+00 -4.48754665e+00 -5.68003219e+00 -5.61738472e+00]
 [-5.33933783e+00 -3.20451075e+00 -5.32851836e+00 -5.26106517e+00]
 [-4.61555908e+00 -2.37440252e+00 -4.37678988e+00 -3.65109827e+00]
 [-3.70640087e+00 -4.632571